## Notebook para testes

In [1]:
import base64

import pandas as pd
import os
import requests
import streamlit as st
from PIL import Image

import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from datetime import datetime

In [2]:
def get_df_from_csv(csv_name):
    return pd.read_csv(download_csv_file(csv_name), index_col=0)


def download_csv_file(csv_name):
    req = requests.get(
        f"https://raw.githubusercontent.com/pythonbrasil/dados/main/dados/python-brasil-2020/{csv_name}.csv"
    )
    
    if not os.path.exists('./files'):
        os.makedirs('./files')    
    csv_name = f"./files/{csv_name}.csv"
    url_content = req.content
    with open(csv_name, "wb") as csv_file:
        csv_file.write(url_content)
    csv_file.close()
    return csv_name

In [3]:
def get_event_tickets(csv):
    df = get_df_from_csv(csv).reset_index()
    df['date'] = pd.to_datetime(df['interval_start_timestamp']).dt.tz_convert(tz='America/Sao_Paulo').dt.date
    return df[df['date'] >= datetime(2018, 11, 2).date()][['date','messages']]

In [4]:
texto_df = get_event_tickets("discord-atividade-por-texto")
texto_df.messages.sum()

19223

In [73]:
def get_event_tickets():
    df = get_df_from_csv("inscrições-palestras")

    rename_columns = {
        "Orientação sexual:": "Orientação sexual",
        "Se outro, qual?": "Se define - Se outro, qual?",
        "Se outro, qual?.1": "Se identifica - Se outro, qual?",
        "Se outro, qual?.2": "Orientação sexual - Se outro, qual?",
        "Se sim, qual?": "Necessidades específicas - Se sim, qual?",
        "Se sim, qual?.1": "comunidade local - Se sim, qual?",
    }

    return (
        df[
            [
                "Como você se define",
                "Se outro, qual?",
                "Como você se identifica?",
                "Se outro, qual?.1",
                "Faz parte da população T (pessoa transgênera, travesti)?",
                "Orientação sexual:",
                "Se outro, qual?.2",
                "Pessoa com necessidades específicas?",
                "Se sim, qual?",
                "Em qual UF você reside?",
                "Em qual país você reside?",
                "De quais edições da Python Brasil você já participou?",
                "Você já participou de algum outro evento Python?",
                "Você participou de outros eventos online durante a pandemia de covid-19?",
                "Há quanto tempo você programa em Python?",
                "Como você classificaria seu nível de conhecimento em Python?",
                "Você é estudante?",
                "Você trabalha com Python?",
                "Você pretende submeter alguma atividades (palestra, tutorial) para a Python Brasil?",
                "Você faz parte de alguma comunidade local? (grupy, PUG, PyLadies, AfroPython)",
                "Se sim, qual?.1",
            ]
        ]
        .rename(columns=rename_columns)
        .fillna("N/A")
    )

In [90]:
df = get_event_tickets()
col = "Em qual país você reside?"
df[col] = df[col].str.strip()
df.loc[df[col] == 'Por favor, que formulário mais sem sentido. Querem também saber que orientação sexual vou querer ter na próxima encarnação? Isso não tem sentido.',col] = 'N/A'
df.loc[df[col] == ',',col] = 'N/A'
df.loc[df[col] == 'PNR',col] = 'N/A'
df.loc[df[col] == 'Prefiro não responder.',col] = 'N/A'
df.loc[df[col] == 'Alemanha, Berlin',col] = 'Alemanha'
df.loc[df[col] == 'Canada',col] = 'Canadá'
df.loc[df[col] == 'Dublin, Irlanda',col] = 'Irlanda'
df.loc[df[col] == 'Ireland',col] = 'Irlanda'
df.loc[df[col] == 'irlanda',col] = 'Irlanda'
df.loc[df[col] == 'Italy',col] = 'Itália'
df.loc[df[col] == 'Mocambique',col] = 'Moçambique'
df.loc[df[col] == 'Países Baixo',col] = 'Países Baixos'
df.loc[df[col] == 'Perú',col] = 'Peru'
df.loc[df[col] == 'US',col] = 'EUA'
df.loc[df[col] == 'USA',col] = 'EUA'
df.loc[df[col] == 'Usa',col] = 'EUA'
df.loc[df[col] == 'Estados Unidos',col] = 'EUA'
df.loc[df[col] == 'United States',col] = 'EUA'
df.loc[df[col] == 'CZ',col] = 'República Tcheca'
df.loc[df[col] == 'United Kingdom',col] = 'Reino Unido'
df.loc[df[col] == 'colombia',col] = 'Colombia'
df.loc[df[col] == 'portugal',col] = 'Portugal'
df.loc[df[col] == 'PT',col] = 'Portugal'
df.loc[df[col] == 'thailand',col] = 'Tailandia'
df.loc[df[col] == 'France',col] = 'França'
df.loc[df[col] == 'Japao',col] = 'Japão'

In [91]:
x= list(df[col].unique())
x.sort()
x

['Alemanha',
 'Argentina',
 'Austria',
 'Bangladesh',
 'Bélgica',
 'Cabo verde',
 'Canadá',
 'Chile',
 'Colombia',
 'EUA',
 'Espanha',
 'França',
 'Holanda',
 'Hungria',
 'India',
 'Irlanda',
 'Itália',
 'Japão',
 'Luxemburgo',
 'Marte',
 'Moçambique',
 'México',
 'N/A',
 'Noruega',
 'Paraguay',
 'Países Baixos',
 'Peru',
 'Portugal',
 'Reino Unido',
 'República Tcheca',
 'Suécia',
 'Tailandia',
 'Uruguay']

In [2]:
x= pd.read_csv('../files/inscrições-palestras-2021.csv')

In [3]:
rename_columns = {
                "como_voce_se_define": "Como você se define",
                "como_voce_se_define_se_outro": "Se outro, qual?",
                "como_voce_se_indentifica": "Como você se identifica?",
                "como_voce_se_indentifica_se_outro": "Se define - Se outro, qual?",
                "faz_parte_da_populacao_t": "Faz parte da população T (pessoa transgênera, travesti)?",
                "orientaca_sexual": "Orientação sexual",
                "orientaca_sexual_se_outro": "Se identifica - Se outro, qual?",
                "pessoa_com_necessidade_especifica":"Pessoa com necessidades específicas?",
                "pessoa_com_necessidade_especifica_se_sim_qual":"Necessidades específicas - Se sim, qual?",
                "uf": "Necessidades específicas - Se sim, qual?",
                "pais": "Em qual país você reside?",
                "edicoes_anteriores_pybr": "De quais edições da Python Brasil você já participou?",
                "eventos_pybr": "Você já participou de algum outro evento Python?",
                "eventos_online": "Você participou de outros eventos online durante a pandemia de covid-19?",
                "programa_em_python": "Há quanto tempo você programa em Python?",
                "nivel_python": "Como você classificaria seu nível de conhecimento em Python?",
                "estudante": "Você é estudante?",
                "trabalha_com_python": "Você trabalha com Python?",
                "parte_de_alguma_comunidade": "Você faz parte de alguma comunidade local? (grupy, PUG, PyLadies, AfroPython)",
                "comunidade_qual":"Comunidade local - Se sim, qual?"
            }

In [4]:
x= x.rename(columns=rename_columns).fillna("N/A")

In [8]:
x['Você já participou de algum outro evento Python?'].unique()

array(['Não, esse é meu primeiro evento da linguagem Python / No, es mi primer evento de la comunidad Python',
       'Sim, eventos locais / Sí, eventos locales',
       'Sim, eventos locais / Sí, eventos locales | Sim, eventos regionais (Python Nordeste, Python Cerrado, Python Sul, etc) / Sí, eventos regionales (PyCon Latam, PyCon Bolivia)',
       'Sim, eventos locais / Sí, eventos locales | Sim, eventos regionais (Python Nordeste, Python Cerrado, Python Sul, etc) / Sí, eventos regionales (PyCon Latam, PyCon Bolivia) | Sim, eventos internacionais / Sí, eventos internacionales',
       'Sim, eventos regionais (Python Nordeste, Python Cerrado, Python Sul, etc) / Sí, eventos regionales (PyCon Latam, PyCon Bolivia)',
       'Sim, eventos internacionais / Sí, eventos internacionales',
       'Sim, eventos locais / Sí, eventos locales | Sim, eventos internacionais / Sí, eventos internacionales',
       'Sim, eventos regionais (Python Nordeste, Python Cerrado, Python Sul, etc) / Sí, eventos